In [7]:
import cv2
import numpy as np
import math
from easydict import EasyDict

root = "dataset\\kitti\\"
raw  = root + "object\\training\\"
img2 = raw + "image_2\\"
img3 = raw + "image_3\\"
lidar= raw + "velodyne\\"
calib= raw + "calib\\"
label= raw + "label_2\\"
bev2 = raw + "bev_2\\"

boundary = EasyDict({
    "minX": 0,
    "maxX": 50,
    "minY": -25,
    "maxY": 25,
    "minZ": -2.73,
    "maxZ": 1.27
})
BEV_WIDTH = 608   # across y axis -25m ~ 25m
BEV_HEIGHT = 608  # across x axis 0m ~ 50m
DISCRETIZATION_X = (boundary.maxX - boundary.minX) / BEV_HEIGHT
DISCRETIZATION_Y = (boundary.maxY - boundary.minY) / BEV_HEIGHT

In [8]:
DEG2RAD = 0.01745329252
CAMERA_POS_Y = 0  # d (cm)
CAMERA_POS_X = 0  # l (cm)
CAMERA_POS_Z = 173# h (cm)
FOV_H = 54.0      # (degree)
FOV_V = 54.0      # (degree)
DST_REMAPPED_WIDTH  = 608
DST_REMAPPED_HEIGHT = 608

def build_ipm_table(
    srcw: int, srch: int, dstw: int, dsth: int,
    vptx: int, vpty: int):
    maptable = np.zeros((dstw*dsth,), dtype=int)
    
    alpha_h = 0.5 * FOV_H * DEG2RAD
    alpha_v = 0.5 * FOV_V * DEG2RAD
    gamma = -(vptx - (srcw >> 1)) * alpha_h / (srcw >> 1) # camera pan angle
    theta = -(vpty - (srch >> 1)) * alpha_v / (srch >> 1) # camera tilt angle

    front_map_start_position = dsth >> 1
    front_map_end_position = front_map_start_position + dsth
    side_map_mid_position = dstw >> 1

    # scale to get better mapped image
    front_map_scale_factor = 4
    side_map_scale_factor  = 2

    for y in range(dstw):
        for x in range(front_map_start_position, front_map_end_position):
            idx = y * dsth + (x - front_map_start_position)
            deltax = front_map_scale_factor * (front_map_end_position - x - CAMERA_POS_X)
            deltay = side_map_scale_factor * (y - side_map_mid_position - CAMERA_POS_Y)

            if deltay == 0:
                maptable[idx] = maptable[idx - dsth]
            else:
                atan_deltay_deltax = math.atan(deltay / deltax)
                u = (int)(math.atan(CAMERA_POS_Z * math.sin(atan_deltay_deltax / deltay) - (theta - alpha_v)) / (2 * alpha_v / srch))
                v = (int)((atan_deltay_deltax - (gamma - alpha_h)) / (2 * alpha_h / srcw))
                if 0 <= u < srch and 0 <= v < srcw:
                    maptable[idx] = srcw * u + v
                else:
                    maptable[idx] = -1
    return maptable

def inverse_perspective_mapping(
    dstw: int, dsth: int, src, maptable: list):
    dst = np.zeros((dsth*dstw, 3), dtype = "uint8")
    src = src.reshape((-1, 3))
    # dst image (1cm/pixel)
    idx = 0
    for j in range(dsth):
        for i in range(dstw):
            if maptable[idx] != -1:
                dst[i * dsth + j] = src[maptable[idx]]
            else:
                dst[i * dsth + j] = [0, 0, 0]
            idx+=1
    return dst

In [9]:
for sample_id in range(7481): # train+val / 7518 test
    image_name = f"{sample_id:06d}.png"
    # lidar_name = f"{sample_id:06d}.bin"
    # calib_name = f"{sample_id:06d}.txt"
    # label_name = f"{sample_id:06d}.txt"
    cim2 = cv2.imread(img2+image_name, cv2.IMREAD_COLOR)

    SRC_RESIZED_WIDTH   = cim2.shape[1]
    SRC_RESIZED_HEIGHT  = cim2.shape[0]

    # init vanishing point at center of image
    vanishing_point_x = SRC_RESIZED_WIDTH  >> 1
    vanishing_point_y = SRC_RESIZED_HEIGHT >> 1

    # build inverse perspective mapping table first
    ipm_table = build_ipm_table(SRC_RESIZED_WIDTH, SRC_RESIZED_HEIGHT, 
                            DST_REMAPPED_WIDTH, DST_REMAPPED_HEIGHT, 
                            vanishing_point_x, vanishing_point_y)


    # cv2.resize(img_bev, (configs.img_size, configs.img_size))
    cim2 = cv2.imread(img2+image_name, cv2.IMREAD_COLOR)
    # cim2_res = cv2.resize(cim2, (SRC_RESIZED_WIDTH, SRC_RESIZED_HEIGHT))
    # cim2_gray = cv2.cvtColor(cim2_res, CV_BGR2GRAY)

    img_mapped = inverse_perspective_mapping(DST_REMAPPED_WIDTH, DST_REMAPPED_HEIGHT, cim2, ipm_table)
    img_mapped = img_mapped.reshape((DST_REMAPPED_HEIGHT, DST_REMAPPED_WIDTH, 3))
    img_mapped = cv2.rotate(img_mapped, cv2.ROTATE_180)

    # line(imresize, Point(vanishing_point_x + 10, vanishing_point_y), Point(vanishing_point_x - 10, vanishing_point_y), Scalar(0, 0, 255));
    # line(imresize, Point(vanishing_point_x, vanishing_point_y + 10), Point(vanishing_point_x, vanishing_point_y - 10), Scalar(0, 0, 255));
    # imshow("resize", imresize);
    # cv2.imshow("remap", img_mapped); cv2.waitKey(0); cv2.destroyAllWindows()
    cv2.imwrite(bev2+image_name, img_mapped)